## 3D


#### Problem Setup

$0 = u_{xx} + u_y + \phi u_t$

For the generation of our training data we use:

$u: \mathbb{R}^3 \rightarrow \mathbb{R}, \; u(x,y,t) = x + y + t$ <br>
$f: \mathbb{R}^3 \rightarrow \mathbb{R}, \;f(x,y,t) = 0$ <br>
$X_i := (x_i, y_i, t_i) \in [0,1] \times [0,1] \times [0, 0.135] \subseteq \mathbb{R}^3$ for $i \in \{1, \dotsc, n\}$

and our known function values will be $\{u(X_i), f(X_i)\}_{i \in \{1, \dotsc, n\}}$.

We assume that $u$ can be represented as a Gaussian process with SE kernel:

$u \sim \mathcal{GP}(0, k_{uu}(X_i, X_j; \theta))$, where $\theta = \{\sigma, l_x, l_y, l_t\}$.

Set the linear operator to:

$\mathcal{L}_X^{\phi} := \partial_{xx} + \partial_y + \phi \partial_t$

so that

$\mathcal{L}_X^{\phi} u = f$

Problem at hand: Estimate $\phi$ (we expect $\phi = -1$).


#### Step 1: Simulate data

In [15]:
import time
import numpy as np
import sympy as sp
from scipy.linalg import solve_triangular
import scipy.optimize as opt

In [16]:
# Global variables: x, y, t, n, y_u, y_f, s

*Parameters, that can be modified:*

In [17]:
# Number of data samples:
n = 20

# Noise parameter:
s = 1e-7

In [18]:
def simulate_data():
    x = np.random.rand(n)
    y = np.random.rand(n)
    t = np.array([0.015*np.random.randint(10) for i in range(n)])
    y_u = x + y + t
    y_f = 0*np.ones(n)
    return (x,y,t,y_u,y_f)
(x,y,t,y_u,y_f) = simulate_data()

#### Step 2: Evaluate kernels

$k_{uu}(X_i, X_j; \theta) = \sigma \cdot exp(-\frac{1}{2l_x}(x_i-x_j)^2 - \frac{1}{2l_y}(y_i-y_j)^2 - \frac{1}{2l_t}(t_i-t_j)^2)$

In [19]:
x_i, x_j, y_i, y_j, t_i, t_j, sigma, l_x, l_y, l_t, phi = sp.symbols('x_i x_j y_i y_j t_i t_j sigma l_x l_y l_t phi')
kuu_sym = sigma*sp.exp(-1/(2*l_x)*((x_i - x_j)**2) - 1/(2*l_y)*((y_i - y_j)**2) - 1/(2*l_t)*((t_i - t_j)**2))
kuu_fn = sp.lambdify((x_i, x_j, y_i, y_j, t_i, t_j, sigma, l_x, l_y, l_t), kuu_sym, "numpy")
def kuu(x, y, t, sigma, l_x, l_y, l_t):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            k[i,j] = kuu_fn(x[i], x[j], y[i], y[j], t[i], t[j], sigma, l_x, l_y, l_t)
    return k

$k_{ff}(X_i,X_j;\theta,\phi) \\
= \mathcal{L}_{X_i}^{\phi} \mathcal{L}_{X_j}^{\phi} k_{uu}(X_i, X_j; \theta) \\
= \partial_{x_i, x_j}k_{uu} + \partial_{y_i, x_j}k_{uu} + \phi \partial_{t_i, x_j}k_{uu} \\
+ \partial_{x_i, y_j}k_{uu} + \partial_{y_i, y_j}k_{uu} + \phi \partial_{t_i, y_j}k_{uu} \\
+ \phi \partial_{x_i, t_j}k_{uu} + \phi \partial_{y_i, t_j}k_{uu} + \phi^2 \partial_{t_i, t_j}k_{uu}$

In [20]:
kff_sym = sp.diff(kuu_sym, x_i, x_i, x_j, x_j) \
        + sp.diff(kuu_sym, y_i, x_j, x_j) \
        + phi*sp.diff(kuu_sym, t_i, x_j, x_j) \
        + sp.diff(kuu_sym, x_i, x_i, y_j) \
        + sp.diff(kuu_sym, y_i, y_j) \
        + phi*sp.diff(kuu_sym, t_i, y_j) \
        + phi*sp.diff(kuu_sym, x_i, x_i, t_j) \
        + phi*sp.diff(kuu_sym, y_i, t_j) \
        + phi**2*sp.diff(kuu_sym, t_i, t_j)
kff_fn = sp.lambdify((x_i, x_j, y_i, y_j, t_i, t_j, sigma, l_x, l_y, l_t, phi), kff_sym, "numpy")
def kff(x, y, t, sigma, l_x, l_y, l_t, phi):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            k[i,j] = kff_fn(x[i], x[j], y[i], y[j], t[i], t[j], sigma, l_x, l_y, l_t, phi)
    return k

$k_{fu}(X_i,X_j;\theta,\phi) \\
= \mathcal{L}_{X_i}^{\phi} k_{uu}(X_i, X_j; \theta) \\
= \partial_{x_i}k_{uu} + \partial_{y_i}k_{uu} + \phi \partial_{t_i}k_{uu}$

In [21]:
kfu_sym = sp.diff(kuu_sym, x_i, x_i) \
        + sp.diff(kuu_sym, y_i) \
        + phi*sp.diff(kuu_sym, t_i) 
kfu_fn = sp.lambdify((x_i, x_j, y_i, y_j, t_i, t_j, sigma, l_x, l_y, l_t, phi), kfu_sym, "numpy")
def kfu(x, y, t, sigma, l_x, l_y, l_t, phi):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            k[i,j] = kfu_fn(x[i], x[j], y[i], y[j], t[i], t[j], sigma, l_x, l_y, l_t, phi)
    return k

In [22]:
def kuf(x, y, t, sigma, l_x, l_y, l_t, phi):
    return kfu(x, y, t, sigma, l_x, l_y, l_t, phi).T

#### Step 3: Computing the negative log-likelihood (with block matrix inversion, Cholesky decomposition, potentially SVD)

In [23]:
def nlml(params):
    
    sigma_exp = np.exp(params[0])
    l_x_exp = np.exp(params[1])
    l_y_exp = np.exp(params[2]) 
    l_t_exp = np.exp(params[3])
    # phi = params[4]
    
    A = kuu(x, y, t, sigma_exp, l_x_exp, l_y_exp, l_t_exp) + s*np.eye(n)
    B = kfu(x, y, t, sigma_exp, l_x_exp, l_y_exp, l_t_exp, params[4]).T
    C = kff(x, y, t, sigma_exp, l_x_exp, l_y_exp, l_t_exp, params[4]) + s*np.eye(n)
    
    # Inversion of A
    A_inv = np.zeros((n, n))
    
    try:
        L = np.linalg.cholesky(A)
        L_inv = solve_triangular(L, np.identity(n), lower=True) # Slight performance boost over np.linalg.inv
        A_inv = L_inv.T @ L_inv
        logdet_A = 2*np.log(np.abs(np.diag(L))).sum()
    except np.linalg.LinAlgError:
        # Inverse of K via SVD
        u, s_mat, vt = np.linalg.svd(A)
        A_inv = vt.T @ np.linalg.inv(np.diag(s_mat)) @ u.T
        logdet_A = np.log(s_mat).sum()
        
    # Inversion of $C-B^T A^{-1} B$
    KA_inv = np.zeros((n, n))
    KA = C - B.T @ A_inv @ B
    
    try:
        L = np.linalg.cholesky(KA)
        L_inv = solve_triangular(L, np.identity(n), lower=True) # Slight performance boost over np.linalg.inv
        KA_inv = L_inv.T @ L_inv
        logdet_KA = 2*np.log(np.abs(np.diag(L))).sum()
    except np.linalg.LinAlgError:
        # Inverse of K via SVD
        u, s_mat, vt = np.linalg.svd(KA)
        KA_inv = vt.T @ np.linalg.inv(np.diag(s_mat)) @ u.T
        logdet_KA = np.log(s_mat).sum()
        
    # Piecing it together
    T = A_inv @ B @ KA_inv
    yKy = y_u @ (A_inv + T @ B.T @ A_inv) @ y_u - 2*y_u @ T @ y_f + y_f @ KA_inv @ y_f
    
    return yKy + logdet_A + logdet_KA

#### Step 4: Optimize hyperparameters

**1. Nelder-Mead**

In [24]:
def callbackf(params):
    print(params)

In [27]:
def minimize_restarts(x,y,y_u,y_f,n=5): 
    all_results = []
    for it in range(0,n):
        all_results.append(opt.minimize(nlml, np.random.rand(5), callback = callbackf, method="Nelder-Mead", 
                                        options={'maxfev':5000, 'fatol':0.001, 'xatol':0.001}))
    return min(all_results, key = lambda x: x.fun)

In [28]:
t0 = time.time()
m = minimize_restarts(x, y, y_u, y_f, 3)
t_Nelder = time.time() - t0
print(m)

[0.1684683  0.75239479 0.76591074 0.66126914 0.61323606]
[0.1684683  0.75239479 0.76591074 0.66126914 0.61323606]
[0.1684683  0.75239479 0.76591074 0.66126914 0.61323606]
[0.17894932 0.73768129 0.76002141 0.63769457 0.57398895]
[0.17894932 0.73768129 0.76002141 0.63769457 0.57398895]
[0.17676728 0.61577327 0.78038065 0.63234484 0.63082295]
[0.17676728 0.61577327 0.78038065 0.63234484 0.63082295]
[0.17676728 0.61577327 0.78038065 0.63234484 0.63082295]
[0.20317181 0.49756899 0.77995    0.61376717 0.5832833 ]
[0.20317181 0.49756899 0.77995    0.61376717 0.5832833 ]
[0.20238362 0.25046556 0.81957538 0.65052015 0.64959727]
[0.20238362 0.25046556 0.81957538 0.65052015 0.64959727]
[0.20238362 0.25046556 0.81957538 0.65052015 0.64959727]
[ 0.24731225 -0.10089755  0.85469404  0.61175428  0.57084549]
[ 0.24731225 -0.10089755  0.85469404  0.61175428  0.57084549]
[ 0.24798002 -0.53043645  0.93756852  0.6177298   0.68943103]
[ 0.24798002 -0.53043645  0.93756852  0.6177298   0.68943103]
[ 0.3003907

[ 0.70223718 -2.21755161  1.39509834 -0.01181892 -1.0778607 ]
[ 0.75058223 -2.10743716  0.93431573  0.09030935 -1.0141399 ]
[ 0.75058223 -2.10743716  0.93431573  0.09030935 -1.0141399 ]
[ 0.78115934 -1.88324423  0.61081708  0.10741055 -1.03801778]
[ 0.78115934 -1.88324423  0.61081708  0.10741055 -1.03801778]
[ 0.78115934 -1.88324423  0.61081708  0.10741055 -1.03801778]
[ 0.78115934 -1.88324423  0.61081708  0.10741055 -1.03801778]
[ 0.78115934 -1.88324423  0.61081708  0.10741055 -1.03801778]
[ 0.80881974 -1.66502262  0.18546224  0.14912241 -0.96833412]
[ 0.80881974 -1.66502262  0.18546224  0.14912241 -0.96833412]
[ 0.73894342 -1.83234862  0.9334113  -0.02930397 -1.12011408]
[ 0.73894342 -1.83234862  0.9334113  -0.02930397 -1.12011408]
[ 0.8505389  -1.21741629 -0.35122383  0.14660912 -1.01220178]
[ 0.86969817 -0.96003589 -0.60247911  0.09482504 -1.06040933]
[ 0.80449559 -1.29473951  0.12679704  0.01495323 -1.11809831]
[ 0.80449559 -1.29473951  0.12679704  0.01495323 -1.11809831]
[ 0.7998

[ 0.13620574  2.33342634  2.00877605  2.1710452  -1.00446224]
[ 0.13620574  2.33342634  2.00877605  2.1710452  -1.00446224]
[ 0.13620574  2.33342634  2.00877605  2.1710452  -1.00446224]
[ 0.13620574  2.33342634  2.00877605  2.1710452  -1.00446224]
[ 0.12479289  2.40631477  2.09124694  2.49803861 -1.00451331]
[ 0.14950068  2.33299734  1.96388404  2.39195704 -1.00296812]
[ 0.1275107   2.39475671  2.06415566  2.39648814 -1.00554313]
[ 0.13723809  2.29358631  2.06414645  2.34603635 -1.00397065]
[ 0.12124626  2.393713    2.10836974  2.42661764 -1.00367935]
[ 0.13413173  2.34884998  2.03356831  2.29143638 -1.00429857]
[ 0.12935919  2.37954772  2.06903589  2.43427286 -1.00430264]
[ 0.13969894  2.34754404  2.01586962  2.38546366 -1.00366349]
[ 0.13969894  2.34754404  2.01586962  2.38546366 -1.00366349]
[ 0.13405493  2.33112887  2.06087527  2.3654599  -1.00405604]
[ 0.12733988  2.37493381  2.07823746  2.39963478 -1.00394805]
[ 0.12733988  2.37493381  2.07823746  2.39963478 -1.00394805]
[ 0.1310

[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.1312

[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.13127861  2.35997178  2.06040048  2.38111708 -1.00395351]
[ 0.1312

...

[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.1082

[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.1082

[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.10829931  1.93481513  0.41324046  0.06432306 -0.99915082]
[ 0.1082

[ 2.36788713 -0.39266715 -2.87801114  0.02527245  0.32943683]
[ 2.36788713 -0.39266715 -2.87801114  0.02527245  0.32943683]
[ 2.36788713 -0.39266715 -2.87801114  0.02527245  0.32943683]
[ 2.36788713 -0.39266715 -2.87801114  0.02527245  0.32943683]
[ 2.36788713 -0.39266715 -2.87801114  0.02527245  0.32943683]
[ 2.36788713 -0.39266715 -2.87801114  0.02527245  0.32943683]
[ 2.36788713 -0.39266715 -2.87801114  0.02527245  0.32943683]
[ 2.36788713 -0.39266715 -2.87801114  0.02527245  0.32943683]
[ 2.39617272 -0.40807327 -2.93164026  0.02575005  0.33590496]
[ 2.39636956 -0.41132565 -2.93783958  0.02571292  0.33249527]
[ 2.39636956 -0.41132565 -2.93783958  0.02571292  0.33249527]
[ 2.39636956 -0.41132565 -2.93783958  0.02571292  0.33249527]
[ 2.37555519 -0.40047822 -2.89683393  0.0255799   0.33198069]
[ 2.38098723 -0.39943497 -2.90466469  0.02547274  0.33118065]
[ 2.38080527 -0.39727883 -2.91448881  0.02535261  0.32412891]
[ 2.38080527 -0.39727883 -2.91448881  0.02535261  0.32412891]
[ 2.3808

[-0.60134365  1.52125626  1.05935057 -0.02593483 -1.0014696 ]
[-0.60134365  1.52125626  1.05935057 -0.02593483 -1.0014696 ]
[-0.59767447  1.52025425  1.05129936 -0.02592393 -1.0017399 ]
[-0.60257874  1.52198186  1.06114066 -0.0259552  -1.00198918]
[-0.60257874  1.52198186  1.06114066 -0.0259552  -1.00198918]
[-0.60180791  1.52131598  1.06031472 -0.02593817 -1.00154784]
[-0.60180791  1.52131598  1.06031472 -0.02593817 -1.00154784]
[-0.60126266  1.52144518  1.05858705 -0.02594857 -1.00197824]
[-0.60126266  1.52144518  1.05858705 -0.02594857 -1.00197824]
[-0.60122398  1.52124633  1.05900276 -0.02593927 -1.00163727]
[-0.60122398  1.52124633  1.05900276 -0.02593927 -1.00163727]
[-0.60122398  1.52124633  1.05900276 -0.02593927 -1.00163727]
[-0.60122398  1.52124633  1.05900276 -0.02593927 -1.00163727]
[-0.60087684  1.52123274  1.05804754 -0.02594406 -1.00186147]
[-0.60087684  1.52123274  1.05804754 -0.02594406 -1.00186147]
[-0.60087684  1.52123274  1.05804754 -0.02594406 -1.00186147]
[-0.5993

In [29]:
t_Nelder

264.58030796051025

In [30]:
print('The inferred parameter is:', m.x[4])

The inferred parameter is: -1.004052962023918
